In [1]:
import torch
import pandas as pd
import os

In [2]:
!pip install clean-text
!pip install langdetect
#!pip install validators

In [3]:
from gptzero.model import GPT2PPL
from cleantext import clean
from langdetect import detect
#import validators

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [4]:
import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [5]:
def contains_alpha(input):
    search_results = re.search('[a-zA-Z]', input)
    if search_results == None:
        return False
    else:
        return True
    return search_results
def isurl(input):   
    regex = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
        r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
    return re.match(regex, input) is not None 

def printline():
    print("------------------------------------------")
    
def infer(sentence):
    sentence1 = sentence
    sentence = remove_emojis(sentence)
    sentence = clean(sentence, no_emoji=True)
    if sentence != sentence1:
        print(sentence1, sentence)
    model = GPT2PPL()
    output = model(sentence)
    return output[0]["label"]

def enhanced_detect(input):
    printline()
    print(input)
    printline()
    return detect(input)

In [6]:
df = pd.read_csv("full_frame.csv", sep='\t', encoding='utf-8')

In [7]:
df.head()

,utterances,human
0,Yeah but that's not the detail I am looking fo...,1
1,"Mais tu veux dire ""on est"", si même si tu voul...",1
2,U should check out the stuff that Jordan Peter...,0
3,And here are the rules of the game.\n\n>The ob...,1
4,That's not the point.,1


In [8]:
testsent = "The fitnessgram pacer test is an aerobic capacity test designed to get progressively more difficult as it continues."
infer(testsent)
df.describe()


The fitnessgram pacer test is an aerobic capacity test designed to get progressively more difficult as it continues. the fitnessgram pacer test is an aerobic capacity test designed to get progressively more difficult as it continues.
end 21
sum tensor(inf, device='cuda:0')
end 21
sum tensor(inf, device='cuda:0')


,human
count,307046.000000
mean,0.403848
std,0.490669
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [9]:
#Clean the Data
df = df.dropna(subset=['utterances'])
df = df[df.utterances != '']
df["contains_alpha"] = df["utterances"].map(contains_alpha)
df = df[df.contains_alpha]
df["isurl"] = df["utterances"].map(isurl)
df = df[~df.isurl]
#df["language"] = df["utterances"].map(enhanced_detect)

#df.describe()

In [ ]:
df["predictions"] = df["utterances"].map(infer)
df.to_csv("gpt_zero_results.csv", sep='\t', encoding='utf-8',index=False)

Yeah but that's not the detail I am looking for in a random movie detail. yeah but that's not the detail i am looking for in a random movie detail.
end 17
sum tensor(5.7522e+30, device='cuda:0')
end 17
sum tensor(5.7522e+30, device='cuda:0')
Mais tu veux dire "on est", si même si tu voulies les voir dans la tâche... mais tu veux dire "on est", si meme si tu voulies les voir dans la tache...
end 26
sum tensor(inf, device='cuda:0')
end 26
sum tensor(inf, device='cuda:0')
U should check out the stuff that Jordan Peterson says about serotonin in the nervous systems of lobsters. 

I'm inclined to say that ur assertion here is less salient than u think it is. u should check out the stuff that jordan peterson says about serotonin in the nervous systems of lobsters.
i'm inclined to say that ur assertion here is less salient than u think it is.
end 41
sum tensor(inf, device='cuda:0')
end 22
sum tensor(inf, device='cuda:0')
end 18
sum tensor(5.8102e+34, device='cuda:0')
And here are the rules of

end 306
sum tensor(inf, device='cuda:0')
end 24
sum tensor(inf, device='cuda:0')
end 52
sum tensor(inf, device='cuda:0')
end 67
sum tensor(inf, device='cuda:0')
end 29
sum tensor(inf, device='cuda:0')
end 39
sum tensor(inf, device='cuda:0')
end 17
sum tensor(1.2692e+37, device='cuda:0')
end 28
sum tensor(inf, device='cuda:0')
end 20
sum tensor(inf, device='cuda:0')
end 17
sum tensor(2.7301e+30, device='cuda:0')
end 13
sum tensor(4.5013e+27, device='cuda:0')
I'm not saying the only thing that's in the relationship is about you. It's the only thing I'd say. I know it's hard for him. i'm not saying the only thing that's in the relationship is about you. it's the only thing i'd say. i know it's hard for him.
end 33
sum tensor(inf, device='cuda:0')
end 16
sum tensor(3.6243e+24, device='cuda:0')
end 9
sum tensor(3.9693e+14, device='cuda:0')
end 8
sum tensor(8.4295e+12, device='cuda:0')
That's a good point, but if I have sex in the middle of the ocean, I'm going to have to make sure to mastur

end 522
sum tensor(inf, device='cuda:0')
end 10
sum tensor(3.9923e+18, device='cuda:0')
end 33
sum tensor(inf, device='cuda:0')
end 52
sum tensor(inf, device='cuda:0')
end 108
sum tensor(inf, device='cuda:0')
end 53
sum tensor(inf, device='cuda:0')
end 28
sum tensor(inf, device='cuda:0')
end 96
sum tensor(inf, device='cuda:0')
end 97
sum tensor(inf, device='cuda:0')
end 13
sum tensor(5.6699e+26, device='cuda:0')
end 31
sum tensor(inf, device='cuda:0')
Always YTA on these wedding posts where you expect people to drop so much money on YOU. Who cares if she has tattoos, it's her money! For my 'bachelorette" I had my friends over for dinner?! We got married in a small wedding (30 people) abroad as my partner and i are from different countries, and made it very clear that we did not expect people to attend OR buy gifts. The ones who wanted to came, the ones who couldn't, didn't. No hard feelings about it either. We also paid for the accommodation for around half the wedding party who we kne

end 220
sum tensor(inf, device='cuda:0')
end 68
sum tensor(inf, device='cuda:0')
end 151
sum tensor(inf, device='cuda:0')
How much Gaia data did you and your team have to analyze to get these results and what kinds of software and equations did you have to use to do it? How much of this type of research is based on computational physics? Thanks! how much gaia data did you and your team have to analyze to get these results and what kinds of software and equations did you have to use to do it? how much of this type of research is based on computational physics? thanks!
end 48
sum tensor(inf, device='cuda:0')
end 33
sum tensor(inf, device='cuda:0')
end 13
sum tensor(2.5187e+21, device='cuda:0')
end 2
sum tensor(72097.3047, device='cuda:0')
It's not event but I don't think if Cannibla will show up at all, because he didn't play a major role in Dance if I remember correctly, and dragon CGI is expensive. it's not event but i don't think if cannibla will show up at all, because he didn't play

end 74
sum tensor(inf, device='cuda:0')
end 74
sum tensor(inf, device='cuda:0')
Do these kinds of anxiety attacks happen often? do these kinds of anxiety attacks happen often?
end 9
sum tensor(4.8902e+18, device='cuda:0')
end 9
sum tensor(4.8902e+18, device='cuda:0')
I have a really big cat on my butt! It is a really cool thing to have. A lot of people don't like cats but for me, it is awesome! I am really happy to share my life with this big ol' cat! And I love when people ask me about my time in the porn industry! Most people are interested to hear about what it is like to work in a big porn studio and have the opportunity to make a lot of money...I love talking about this! i have a really big cat on my butt! it is a really cool thing to have. a lot of people don't like cats but for me, it is awesome! i am really happy to share my life with this big ol' cat! and i love when people ask me about my time in the porn industry! most people are interested to hear about what it is like to w

...

end 111
sum tensor(inf, device='cuda:0')
end 12
sum tensor(3.7169e+19, device='cuda:0')
end 24
sum tensor(inf, device='cuda:0')
end 14
sum tensor(3.4713e+25, device='cuda:0')
end 6
sum tensor(1.0720e+09, device='cuda:0')
end 27
sum tensor(inf, device='cuda:0')
end 11
sum tensor(2.0923e+22, device='cuda:0')
end 15
sum tensor(1.1764e+33, device='cuda:0')
He wants to suck the face of a pig and make a new one himself. he wants to suck the face of a pig and make a new one himself.
end 16
sum tensor(3.5255e+26, device='cuda:0')
end 16
sum tensor(3.5255e+26, device='cuda:0')
Are you ever gonna be able to afford to retire?
I can't find a job for more than minimum wage and I am willing to work hard and have a lot of education are you ever gonna be able to afford to retire?
i can't find a job for more than minimum wage and i am willing to work hard and have a lot of education
end 36
sum tensor(inf, device='cuda:0')
end 11
sum tensor(2.1990e+16, device='cuda:0')
end 24
sum tensor(1.6502e+35, devi

end 17
sum tensor(8.3750e+23, device='cuda:0')
end 17
sum tensor(8.3750e+23, device='cuda:0')
Yes. yes.
end 2
sum tensor(115.5803, device='cuda:0')
end 2
sum tensor(115.5803, device='cuda:0')
What about Puck's new model looks bad? what about puck's new model looks bad?
end 9
sum tensor(1.5009e+28, device='cuda:0')
end 9
sum tensor(1.5009e+28, device='cuda:0')
YTA. if you are going by traditions then traditionally  the bride's family foots the majority of the bill for the wedding. They are not entitled to help out unless they desire to do so. I think it was tacky to ask for help to pay the honeymoon as they didn't even offer money for the wedding to begin with. yta. if you are going by traditions then traditionally the bride's familyfoots the majority of the bill for the wedding. they are not entitled to help out unless they desire to do so. i think it was tacky to ask for help to pay the honeymoon as they didn't even offer money for the wedding to begin with.
end 69
sum tensor(inf, dev

end 71
sum tensor(inf, device='cuda:0')
end 8
sum tensor(7.5853e+20, device='cuda:0')
end 14
sum tensor(2.4177e+35, device='cuda:0')
end 22
sum tensor(inf, device='cuda:0')
end 27
sum tensor(inf, device='cuda:0')
> Ich hab nur nicht gekauft warum er sieht

Wir können von mir aber eine andere Anhau zur Hochschule (mit einem Hochschulbild zur Hochschule) würden. > ich hab nur nicht gekauft warum er sieht
wir konnen von mir aber eine andere anhau zur hochschule (mit einem hochschulbild zur hochschule) wurden.
end 61
sum tensor(inf, device='cuda:0')
end 19
sum tensor(inf, device='cuda:0')
end 41
sum tensor(inf, device='cuda:0')
None, no historical or archaeological evidence the Isrealites were ever in Egypt. none, no historical or archaeological evidence the isrealites were ever in egypt.
end 17
sum tensor(4.8057e+36, device='cuda:0')
end 17
sum tensor(4.8057e+36, device='cuda:0')
Chocolate Mousse. chocolate mousse.
end 5
sum tensor(1.4789e+10, device='cuda:0')
end 5
sum tensor(1.4789e+10,

end 58
sum tensor(inf, device='cuda:0')
end 43
sum tensor(inf, device='cuda:0')
end 14
sum tensor(3.2929e+32, device='cuda:0')
Congratulations, OP! [You're one of today's Lucky 10,000!](https://xkcd.com/1053/) 

The people being mean and ragging on you about not realizing it until now seem to forget that they also once learned these things and it isn't innate knowledge we're all born with. congratulations, op! [you're one of today's lucky 10,000!](https://xkcd.com/1053/)
the people being mean and ragging on you about not realizing it until now seem to forget that they also once learned these things and it isn't innate knowledge we're all born with.
end 67
sum tensor(inf, device='cuda:0')
end 5
sum tensor(9.8275e+16, device='cuda:0')
end 24
sum tensor(6.9578e+35, device='cuda:0')
end 37
sum tensor(inf, device='cuda:0')
You need once in your life accept Jesus as your saviour.

Being raised as Christian or going to church doesn't make you one , you become Christian when you hear the gospe

end 45
sum tensor(inf, device='cuda:0')
end 11
sum tensor(4.1679e+22, device='cuda:0')
end 14
sum tensor(3.3525e+34, device='cuda:0')
end 20
sum tensor(inf, device='cuda:0')
Sick and tired of these spam posts on here, I would like to direct you to the [rules](/r/AskHistorians/wiki/rules) and the [prompts](/r/AskHistorians/search?q=title%3A%22History+of+the+United+States+of+America%22&restrict_sr=on&sort=new&t=all) sidebar to the right. Posts about people's own experiences are not appropriate for this subreddit and should be posted in /r/HistoryWhatIf. sick and tired of these spam posts on here, i would like to direct you to the [rules](/r/askhistorians/wiki/rules) and the [prompts](/r/askhistorians/search?q=title%3a%22history+of+the+united+states+of+america%22&restrict_sr=on&sort=new&t=all) sidebar to the right. posts about people's own experiences are not appropriate for this subreddit and should be posted in /r/historywhatif.
end 118
sum tensor(inf, device='cuda:0')
end 94
sum tensor